<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Agente_nova_saida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas google-api-python-client google-generativeai


In [2]:
import pandas as pd
import time
from IPython.display import display, Markdown
from tabulate import tabulate
from urllib.parse import quote_plus
import google.generativeai as genai # SDK do Gemini
import json # Para parsear a resposta do Gemini
from google.colab import userdata # Para acessar a API key
from datetime import datetime, timedelta # Importar datetime e timedelta

# --- Configuração do Agente e API Key ---
GOOGLE_API_KEY = None
gemini_model_instance = None # Renomeado para clareza

try:
    # O nome padrão para secrets é sem o sufixo _, a menos que você tenha nomeado especificamente assim.
    # Vou usar 'GOOGLE_API_KEY' como padrão, conforme a prática comum.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        display(Markdown("<font color='red'>**Erro Crítico: GOOGLE_API_KEY não encontrada nos Secrets do Colab.** "
                         "Por favor, configure-a com o nome 'GOOGLE_API_KEY'.</font>"))
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        gemini_model_instance = genai.GenerativeModel(
            model_name='gemini-1.5-flash-latest', # Modelo eficiente e capaz
            # A opção response_mime_type pode ser usada com modelos mais recentes
            # e configurações de segurança apropriadas.
            # generation_config=genai.types.GenerationConfig(
            #     response_mime_type="application/json"
            # )
        )
        display(Markdown("✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*"))
except Exception as e:
    display(Markdown(f"<font color='red'>**Erro Crítico ao configurar o SDK do Gemini:** {e}. "
                     "Verifique sua API Key, permissões e se o nome do secret está correto ('GOOGLE_API_KEY'). "
                     "O agente não funcionará sem isso.</font>"))
    # O script poderia parar aqui ou continuar com funcionalidades limitadas se houvesse um fallback.
    # Para este agente, o Gemini é essencial.

# --- Funções Utilitárias ---

def carregar_dados_viagem(caminho_arquivo: str) -> pd.DataFrame | None:
    """
    Carrega os dados de viagem do arquivo CSV e valida as colunas necessárias.
    """
    colunas_requeridas = ['cidade', 'pais', 'data_chegada', 'data_partida', 'hospedagem']
    try:
        df = pd.read_csv(caminho_arquivo)

        # Validar se as colunas requeridas existem
        if not all(col in df.columns for col in colunas_requeridas):
            colunas_faltando = [col for col in colunas_requeridas if col not in df.columns]
            display(Markdown(f"<font color='red'>**Erro: O arquivo `{caminho_arquivo}` não contém as colunas requeridas.** "
                             f"Faltam as seguintes colunas: {', '.join(colunas_faltando)}.</font>"))
            return None

        display(Markdown(f"### Arquivo de Viagem (`{caminho_arquivo}`)"))
        display(df[colunas_requeridas]) # Exibe apenas as colunas relevantes
        return df
    except FileNotFoundError:
        display(Markdown(f"<font color='red'>**Erro: Arquivo `{caminho_arquivo}` não encontrado.** Verifique o nome e o local.</font>"))
        return None
    except Exception as e:
        display(Markdown(f"<font color='red'>**Erro ao carregar o arquivo CSV `{caminho_arquivo}`:** {e}</font>"))
        return None

def gerar_links_pesquisa_google(cidade: str, pais: str) -> tuple[str, str, str]:
    """Gera links de pesquisa úteis para Google Search e Google Maps."""
    query_atracoes = f"principais atrações turísticas em {cidade} {pais}"
    link_google_search = f"https://www.google.com/search?q={quote_plus(query_atracoes)}"
    query_maps_atracoes = f"atrações turísticas em {cidade}, {pais}"
    link_google_maps_atracoes = f"https://www.google.com/maps/search/{quote_plus(query_maps_atracoes)}"
    query_maps_cidade = f"{cidade}, {pais}"
    link_google_maps_cidade = f"https://www.google.com/maps/place/{quote_plus(query_maps_cidade)}"
    return link_google_search, link_google_maps_atracoes, link_google_maps_cidade

# --- Definição do Agente Pesquisador de Atrações ---

class AgentePesquisadorAtracoes:
    """
    Um agente responsável por pesquisar atrações turísticas usando o Gemini
    e fornecer links úteis para pesquisa manual.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def _obter_sugestoes_gemini(self, cidade: str, pais: str, top_n: int = 15) -> list[dict]:
        """
        Ferramenta interna do agente para consultar o Gemini sobre atrações.
        """
        display(Markdown(f"🧠 *Agente consultando Gemini sobre atrações em: **{cidade}, {pais}** (Top {top_n})...*"))

        prompt = f"""
        Você é um assistente de planejamento de viagens altamente especializado e eficiente.
        Sua tarefa é identificar as {top_n} principais e mais recomendadas atrações turísticas para a cidade de {cidade}, localizada em {pais}.

        Critérios para sua seleção:
        1.  **Popularidade e Reconhecimento:** Atrações bem conhecidas e frequentemente visitadas.
        2.  **Qualidade das Avaliações:** Lugares geralmente com avaliações positivas (ex: acima de 4.0/5 estrelas).
        3.  **Relevância Cultural/Histórica:** Locais com significado importante.
        4.  **Diversidade de Experiências:** Inclua uma mistura de tipos, como museus, monumentos, parques, mercados, igrejas/catedrais, mirantes, etc., quando apropriado para a cidade.
        5.  **Singularidade:** Atrações que oferecem uma experiência única ou icônica da cidade/região.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando uma atração,
        contendo EXATAMENTE as seguintes chaves:
        - "nome": (string) O nome oficial e completo da atração.
        - "tipo_principal": (string) A categoria principal da atração (ex: "Museu de Arte", "Catedral Gótica", "Parque Urbano", "Mercado Histórico", "Monumento Nacional", "Mirante Panorâmico"). Seja específico.
        - "descricao_enxuta": (string) Uma descrição concisa e informativa (1-2 frases) que justifique sua inclusão, destacando seus principais atrativos.
        - "avaliacao_geral": (string, opcional) Uma estimativa da avaliação geral se for amplamente conhecida (ex: "Excelente", "Muito Popular", "4.5/5 estrelas"). Use "N/A" se não houver uma estimativa clara.
        - "destaque_principal": (string) O motivo principal pelo qual um turista deveria visitar (ex: "Vistas incríveis da cidade", "Coleção de arte renascentista", "Arquitetura impressionante", "Atmosfera vibrante").

        Exemplo de um item na lista JSON:
        {{
          "nome": "Museu do Louvre",
          "tipo_principal": "Museu de Arte e Antiguidades",
          "descricao_enxuta": "Um dos maiores e mais visitados museus do mundo, lar de obras-primas como a Mona Lisa e a Vênus de Milo.",
          "avaliacao_geral": "4.7/5 estrelas",
          "destaque_principal": "Coleção de arte de renome mundial"
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.4, # Um pouco mais factual, menos aleatório
                    max_output_tokens=3072 # Espaço suficiente para a lista JSON
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            # Tentativa robusta de extrair JSON, mesmo que o modelo adicione ```json ... ```
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para {cidade}.</font>"))
                return []

            atracoes = json.loads(json_text)
            # Garantir que é uma lista, mesmo que o Gemini retorne algo diferente
            if not isinstance(atracoes, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para {cidade}.</font>"))
                 return []

            display(Markdown(f"✅ *Gemini retornou {len(atracoes)} sugestões de atrações para {cidade}.*"))
            return atracoes[:top_n]
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return []
        except Exception as e:
            # Captura de erros mais genéricos, como problemas de API (quota, etc.)
            # ou bloqueios de segurança não esperados.
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return []

    def pesquisar_destino(self, cidade: str, pais: str, top_n_sugestoes: int = 7) -> dict:
        """
        Executa a pesquisa de atrações para um destino específico.
        """
        display(Markdown(f"\n### 🌍 Agente Iniciando Pesquisa para: **{cidade}, {pais}**"))

        links = gerar_links_pesquisa_google(cidade, pais)
        sugestoes_gemini = self._obter_sugestoes_gemini(cidade, pais, top_n=top_n_sugestoes)

        display(Markdown(f"--- Pesquisa para **{cidade}, {pais}** finalizada pelo agente. ---"))
        return {
            "cidade": cidade,
            "pais": pais,
            "link_google_search": links[0],
            "link_google_maps_atracoes": links[1],
            "link_google_maps_cidade": links[2],
            "sugestoes_gemini": sugestoes_gemini
        }

# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
{atracoes_str}

        Critérios para o Roteiro:
        1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
        2.  **Fluxo Lógico:** Organize as visitas de forma sequencial e eficiente.
        3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
        4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
        5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
        6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
        7.  **Formato Diário:** Organize a saída por dia da viagem.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
        contendo EXATAMENTE as seguintes chaves:
        - "dia": (integer) O número do dia da viagem (começando em 1).
        - "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
        - "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
        - "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

        Exemplo de um item na lista JSON:
        {{
          "dia": 1,
          "data": "{data_chegada}",
          "atividades_sugeridas": [
            "Chegada e check-in na hospedagem ({hospedagem})",
            "Passeio exploratório pela vizinhança da hospedagem",
            "Jantar em um restaurante local"
          ],
          "observacoes": "Aproveite para se ambientar na área."
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}


# --- Função Principal de Execução ---

def executar_planejador_viagem(caminho_arquivo_csv: str):
    """
    Carrega os dados, pesquisa atrações para cada destino e monta o roteiro.
    """
    display(Markdown("## ✨ Iniciando Planejador de Viagens ✨"))

    df_viagens = carregar_dados_viagem(caminho_arquivo_csv)
    if df_viagens is None or df_viagens.empty:
        display(Markdown("<font color='red'>**Erro: Não foi possível carregar os dados de viagem ou o arquivo está vazio.** O planejamento não pode continuar.</font>"))
        return None

    if gemini_model_instance is None:
         display(Markdown("<font color='red'>**Erro: O modelo Gemini não foi configurado corretamente.** Não é possível pesquisar atrações ou montar roteiros.</font>"))
         return None

    agente_pesquisador = AgentePesquisadorAtracoes(gemini_model_instance)
    agente_roteirista = MontadorDeRoteiros(gemini_model_instance) # Instancia o agente roteirista AQUI

    resultados_compilados = []

    for index, row in df_viagens.iterrows():
        cidade = row['cidade']
        pais = row['pais']
        data_chegada = row['data_chegada']
        data_partida = row['data_partida']
        hospedagem = row['hospedagem']

        display(Markdown(f"\n--- Processando destino: **{cidade}, {pais}** ---"))

        # 1. Pesquisar atrações
        pesquisa_destino = agente_pesquisador.pesquisar_destino(cidade, pais)

        # 2. Montar roteiro, APENAS se houver sugestões de atrações
        roteiro_destino = {"roteiro": [], "erro": "Sem sugestões de atrações para montar roteiro"}
        if pesquisa_destino and pesquisa_destino.get("sugestoes_gemini"):
             roteiro_destino = agente_roteirista.montar_roteiro(
                 cidade, pais,
                 pesquisa_destino["sugestoes_gemini"],
                 hospedagem,
                 data_chegada,
                 data_partida
             )
        elif pesquisa_destino and pesquisa_destino.get("erro"):
             roteiro_destino["erro"] = f"Erro na pesquisa de atrações: {pesquisa_destino['erro']}"


        # Compilar resultados
        resultados_compilados.append({
            "cidade": cidade,
            "pais": pais,
            "data_chegada": data_chegada,
            "data_partida": data_partida,
            "hospedagem": hospedagem,
            "pesquisa": pesquisa_destino,
            "roteiro": roteiro_destino.get("roteiro"),
            "erro_roteiro": roteiro_destino.get("erro") # Adiciona campo de erro específico para o roteiro
        })

        # Pequena pausa para não sobrecarregar APIs ou o console
        time.sleep(2)

    display(Markdown("\n## ✅ Planejamento de Viagens Concluído ✅"))
    return resultados_compilados


# --- Execução Principal (Exemplo) ---
# Certifique-se de que o arquivo 'europa.csv' está no caminho especificado.
# dados_finais_viagem = executar_planejador_viagem("/content/europa.csv")

# Você pode então inspecionar 'dados_finais_viagem' para ver os resultados.
# Por exemplo, para ver o roteiro da primeira cidade:
# if dados_finais_viagem and dados_finais_viagem[0].get("roteiro"):
#     display(Markdown("### Roteiro Sugerido para a Primeira Cidade:"))
#     for dia in dados_finais_viagem[0]["roteiro"]:
#          display(Markdown(f"**Dia {dia['dia']} ({dia['data']}):**"))
#          for atividade in dia['atividades_sugeridas']:
#              display(Markdown(f"- {atividade}"))
#          if dia.get("observacoes") and dia["observacoes"] != "N/A":
#              display(Markdown(f"*Observações:* {dia['observacoes']}"))
#          display(Markdown("---"))

# Para ver as sugestões de atrações da primeira cidade:
# if dados_finais_viagem and dados_finais_viagem[0].get("pesquisa") and dados_finais_viagem[0]["pesquisa"].get("sugestoes_gemini"):
#      display(Markdown("### Sugestões de Atrações do Gemini para a Primeira Cidade:"))
#      for atracao in dados_finais_viagem[0]["pesquisa"]["sugestoes_gemini"]:
#           display(Markdown(f"- **{atracao['nome']}** ({atracao['tipo_principal']}): {atracao['descricao_enxuta']} (Avaliação: {atracao.get('avaliacao_geral', 'N/A')})"))

✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*

In [3]:
# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
{atracoes_str}

        Critérios para o Roteiro:
        1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
        2.  **Fluxo Lógico:** Organize as visitas de forma sequencial e eficiente.
        3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
        4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
        5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
        6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
        7.  **Formato Diário:** Organize a saída por dia da viagem.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
        contendo EXATAMENTE as seguintes chaves:
        - "dia": (integer) O número do dia da viagem (começando em 1).
        - "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
        - "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
        - "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

        Exemplo de um item na lista JSON:
        {{
          "dia": 1,
          "data": "{data_chegada}",
          "atividades_sugeridas": [
            "Chegada e check-in na hospedagem ({hospedagem})",
            "Passeio exploratório pela vizinhança da hospedagem",
            "Jantar em um restaurante local"
          ],
          "observacoes": "Aproveite para se ambientar na área."
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}

In [4]:
# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
{atracoes_str}

        Critérios para o Roteiro:
        1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
        2.  **Fluxo Lógico:** Organize as visitas de forma sequencial e eficiente.
        3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
        4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
        5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
        6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
        7.  **Formato Diário:** Organize a saída por dia da viagem.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
        contendo EXATAMENTE as seguintes chaves:
        - "dia": (integer) O número do dia da viagem (começando em 1).
        - "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
        - "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
        - "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

        Exemplo de um item na lista JSON:
        {{
          "dia": 1,
          "data": "{data_chegada}",
          "atividades_sugeridas": [
            "Chegada e check-in na hospedagem ({hospedagem})",
            "Passeio exploratório pela vizinhança da hospedagem",
            "Jantar em um restaurante local"
          ],
          "observacoes": "Aproveite para se ambientar na área."
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}

In [5]:
dados_finais_viagem = dados_compilados = executar_planejador_viagem("/content/europa.csv")

## ✨ Iniciando Planejador de Viagens ✨

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Candia n.143 CAP: 00192
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hotel-Pension Wild



--- Processando destino: **Roma, Itália** ---


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

...

✅ *Gemini retornou 7 sugestões de atrações para Viena.*

--- Pesquisa para **Viena, Aústria** finalizada pelo agente. ---

🧠 *Agente montando roteiro para: **Viena, Aústria** (5 dias)...*

✅ *Gemini montou um roteiro com 5 dias para Viena.*


## ✅ Planejamento de Viagens Concluído ✅

In [6]:
if dados_finais_viagem and dados_finais_viagem[0].get("roteiro"):
    display(Markdown("### Roteiro Sugerido para a Primeira Cidade:"))
    for dia in dados_finais_viagem[0]["roteiro"]:
         display(Markdown(f"**Dia {dia['dia']} ({dia['data']}):**"))
         for atividade in dia['atividades_sugeridas']:
             display(Markdown(f"- {atividade}"))
         if dia.get("observacoes") and dia["observacoes"] != "N/A":
             display(Markdown(f"*Observações:* {dia['observacoes']}"))
         display(Markdown("---"))

if dados_finais_viagem and dados_finais_viagem[0].get("pesquisa") and dados_finais_viagem[0]["pesquisa"].get("sugestoes_gemini"):
     display(Markdown("### Sugestões de Atrações do Gemini para a Primeira Cidade:"))
     for atracao in dados_finais_viagem[0]["pesquisa"]["sugestoes_gemini"]:
          display(Markdown(f"- **{atracao['nome']}** ({atracao['tipo_principal']}): {atracao['descricao_enxuta']} (Avaliação: {atracao.get('avaliacao_geral', 'N/A')})"))

### Roteiro Sugerido para a Primeira Cidade:

**Dia 1 (2025-09-13):**

- Chegada e check-in na hospedagem (Via Candia n.143 CAP: 00192)

- Passeio exploratório pela vizinhança da hospedagem (1-2 horas)

- Jantar em um restaurante local na região do Testaccio (conhecida por sua culinária tradicional).

...

- **Fontana di Trevi** (Fonte Barroca): Uma das fontes mais famosas do mundo, com sua arquitetura exuberante e lendas românticas. (Avaliação: Muito Popular)

- **Basílica de São Pedro** (Catedral Católica Romana): Uma das maiores e mais importantes igrejas do mundo, com uma cúpula imponente e obras de arte renascentistas. (Avaliação: Excelente)

- **Panteão** (Templo Romano e Monumento Histórico): Um templo romano bem preservado, admirado por sua cúpula inovadora e arquitetura impressionante. (Avaliação: Excelente)

- **Piazza Navona** (Praça Histórica e Barroca): Uma animada praça barroca com fontes ornamentadas, artistas de rua e uma atmosfera vibrante. (Avaliação: Muito Popular)

- **Passeio ao longo do Rio Tibre** (Mirante Panorâmico e Caminhada): Um passeio ao longo do rio Tibre oferece vistas panorâmicas da cidade e uma oportunidade de apreciar a atmosfera romana. (Avaliação: N/A)

In [7]:
dados_finais_viagem = dados_compilados = executar_planejador_viagem("/content/europa.csv")

## ✨ Iniciando Planejador de Viagens ✨

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Candia n.143 CAP: 00192
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hotel-Pension Wild



--- Processando destino: **Roma, Itália** ---


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

...

✅ *Gemini retornou 7 sugestões de atrações para Viena.*

--- Pesquisa para **Viena, Aústria** finalizada pelo agente. ---

🧠 *Agente montando roteiro para: **Viena, Aústria** (5 dias)...*

✅ *Gemini montou um roteiro com 5 dias para Viena.*


## ✅ Planejamento de Viagens Concluído ✅

In [8]:
if dados_finais_viagem and dados_finais_viagem[0].get("roteiro"):
    display(Markdown("### Roteiro Sugerido para a Primeira Cidade:"))
    for dia in dados_finais_viagem[0]["roteiro"]:
         display(Markdown(f"**Dia {dia['dia']} ({dia['data']}):**"))
         for atividade in dia['atividades_sugeridas']:
             display(Markdown(f"- {atividade}"))
         if dia.get("observacoes") and dia["observacoes"] != "N/A":
             display(Markdown(f"*Observações:* {dia['observacoes']}"))
         display(Markdown("---"))

if dados_finais_viagem and dados_finais_viagem[0].get("pesquisa") and dados_finais_viagem[0]["pesquisa"].get("sugestoes_gemini"):
     display(Markdown("### Sugestões de Atrações do Gemini para a Primeira Cidade:"))
     for atracao in dados_finais_viagem[0]["pesquisa"]["sugestoes_gemini"]:
          display(Markdown(f"- **{atracao['nome']}** ({atracao['tipo_principal']}): {atracao['descricao_enxuta']} (Avaliação: {atracao.get('avaliacao_geral', 'N/A')})"))

### Roteiro Sugerido para a Primeira Cidade:

**Dia 1 (2025-09-13):**

- Chegada e check-in na hospedagem (Via Candia n.143 CAP: 00192)

- Passeio exploratório pela vizinhança da hospedagem (1-2 horas)

- Jantar em um restaurante local na região do Monti (área próxima à hospedagem).

...

- **Fontana di Trevi** (Fonte Barroca): Uma das fontes mais famosas do mundo, com sua arquitetura exuberante e lenda de jogar uma moeda para garantir o retorno a Roma. (Avaliação: Muito Popular)

- **Panteão** (Templo Romano): Um templo romano antigo excepcionalmente bem preservado, famoso por sua cúpula impressionante e oculus. (Avaliação: Excelente)

- **Piazza Navona** (Praça Barroca): Uma bela praça barroca com fontes ornamentadas, artistas de rua e uma atmosfera vibrante, ideal para passear. (Avaliação: Muito Popular)

- **Basílica de São Pedro** (Catedral Católica Romana): Uma das maiores e mais importantes igrejas do mundo, com uma arquitetura imponente e obras de arte de renome. (Avaliação: Excelente)

- **Espaço Verde: Jardins Borghese e Villa Borghese** (Parque Urbano e Museu de Arte): Um grande parque com galerias de arte, museus, fontes e jardins, oferecendo um refúgio tranquilo no coração de Roma. (Avaliação: Excelente)

In [9]:
print(dados_finais_viagem)

[{'cidade': 'Roma', 'pais': 'Itália', 'data_chegada': '2025-09-13', 'data_partida': '2025-09-16', 'hospedagem': 'Via Candia n.143 CAP: 00192', 'pesquisa': {'cidade': 'Roma', 'pais': 'Itália', 'link_google_search': 'https://www.google.com/search?q=principais+atra%C3%A7%C3%B5es+tur%C3%ADsticas+em+Roma+It%C3%A1lia', 'link_google_maps_atracoes': 'https://www.google.com/maps/search/atra%C3%A7%C3%B5es+tur%C3%ADsticas+em+Roma%2C+It%C3%A1lia', 'link_google_maps_cidade': 'https://www.google.com/maps/place/Roma%2C+It%C3%A1lia', 'sugestoes_gemini': [{'nome': 'Coliseu e Fórum Romano', 'tipo_principal': 'Monumento Nacional e Ruínas Arqueológicas', 'descricao_enxuta': 'Símbolo icônico de Roma, o Coliseu é um anfiteatro antigo, enquanto o Fórum Romano revela os restos da antiga cidade romana.', 'avaliacao_geral': 'Excelente', 'destaque_principal': 'Experiência histórica inesquecível'}, {'nome': 'Museus Vaticanos e Capela Sistina', 'tipo_principal': 'Museu de Arte e Religião', 'descricao_enxuta': 'Lar